In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor


def score_dataset(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score


# Prepare data
df = pd.read_csv(r"C:\Users\mugok\Desktop\Data sets\ames.csv")
X = df.copy()
y = X.pop("SalePrice")

In [3]:
X_1 = pd.DataFrame()  # dataframe to hold new features

X_1["LivLotRatio"] = df['GrLivArea']/df['LotArea']
X_1["Spaciousness"] = (df['FirstFlrSF'] +df['SecondFlrSF'])/df['TotRmsAbvGrd']
X_1["TotalOutsideSF"] = df['WoodDeckSF']+df['OpenPorchSF']+df['EnclosedPorch']+df['Threeseasonporch']+df['ScreenPorch']


In [5]:
# One-hot encode BldgType. Use `prefix="Bldg"` in `get_dummies`
X_2 =pd.get_dummies(df['BldgType'] , prefix = "Bldg") 
# Multiply
X_2 = X_2.mul(df['GrLivArea'] , axis = 0)

In [7]:
#Create a feature PorchTypes that counts how many of the following are greater than 0.0
X_3 = pd.DataFrame()

X_3["PorchTypes"] = (df[['WoodDeckSF',
                         'OpenPorchSF',
                         'EnclosedPorch',
                         'Threeseasonporch'
                         ,'ScreenPorch']]).gt(0.0).sum(axis=1)


In [8]:
#Break Down a Categorical Feature
#MSSubClass describes the type of a dwelling:

df['MSSubClass'].unique()

array(['One_Story_1946_and_Newer_All_Styles', 'Two_Story_1946_and_Newer',
       'One_Story_PUD_1946_and_Newer',
       'One_and_Half_Story_Finished_All_Ages', 'Split_Foyer',
       'Two_Story_PUD_1946_and_Newer', 'Split_or_Multilevel',
       'One_Story_1945_and_Older', 'Duplex_All_Styles_and_Ages',
       'Two_Family_conversion_All_Styles_and_Ages',
       'One_and_Half_Story_Unfinished_All_Ages',
       'Two_Story_1945_and_Older', 'Two_and_Half_Story_All_Ages',
       'One_Story_with_Finished_Attic_All_Ages',
       'PUD_Multilevel_Split_Level_Foyer',
       'One_and_Half_Story_PUD_All_Ages'], dtype=object)

In [9]:
#Create a feature containing only these first words by splitting MSSubClass at the first underscore _.
X_4 = pd.DataFrame()


X_4['MSClass'] = (df['MSSubClass']).str.split('_' , n=1,expand = True)[0]


In [10]:
#Create a feature MedNhbdArea that describes the median of GrLivArea grouped on Neighborhood.
X_5 = pd.DataFrame()


X_5["MedNhbdArea"] = (df.groupby("Neighborhood")['GrLivArea'].transform('median'))


In [15]:
#score the model with all of your new features added.
X_new = X.join([X_1, X_2, X_3, X_4, X_5])
score_dataset(X_new, y)

C:\Users\mugok\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Users\mugok\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
C:\Users\mugok\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
C:\Users\mugok\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  ret

C:\Users\mugok\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Users\mugok\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
C:\Users\mugok\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
C:\Users\mugok\AppData\Roaming\Python\Python310\site-packages\xgboost\data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  ret

0.14039620459068347